In [ ]:
#! pip install google-generativeai

In [1]:
from PIL import Image
import google.generativeai as genai
import os
import shutil
import time
import re
import csv

# Gemini API config
genai.configure(api_key="YOUR API KEY")

e:\Program Files\Anaconda\envs\cuda310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
'''For using this notebook recreate the following structure

parent_dir
-
-- noclass_memes --> memes to be classified
-
--new_hate_speech
-
--new_inappropiate_content
-
--new_none
-
--new_dataset.csv --> outcome dataset'''

# Defining directories
parent_dir = # insert here your parent directory
noclass_memes = parent_dir + "noclass_memes/"
hs_path = parent_dir + "new_hate_speech/"
ic_path = parent_dir + "new_inappropiate_content/"
none_path = parent_dir + "new_none/"
new_dataset_path = parent_dir + "new_dataset.csv"

In [3]:
# Setting up the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]

system_instruction = "You are going to be provided with an image of a meme in spanish and you will be performing three different tasks:\n1. Indicate if the meme is written in English or Spanish in bold\n2. You will need to extract the text of the meme like an OCR, displaying it in an appropiate format for storing it in a database (without newlines, in a sentence). The output for this task will be displayed in the following format: /*HERE GOES THE MEME TEXT*/\n3. You will be performing two classification sub-tasks:\n   A) The first task will be clasifying the meme between the three following categories:\n\t- hate_speech: this category will be composed by memes that contain racism, classism, sexism or other.\n\t- inappropiate_content: this category will contain memes that are inappropiate but do not contain hate speech.\n\t- none: this category will be composed by memes that do not belong to the other categories, memes that you do not understand, memes in a language that is not spanish or any other image that is not a meme.\n   B) The second task will be a finer-grain classification performed only for the memes considered as hate_speech in the previous task, the classes are the following:\n\t- racism: category containing racist memes.\n\t- classism: category containing classist memes.\n\t- sexism: category containing sexist memes.\n\t- other: category containing other type of hate speech memes.\nYou can develop your explanations for the classification, but you should always specify the categories in the following format only at the end of the explanation: ||CATEGORY TASK A||\t&&CATEGORY TASK B&&\nUnder no circumstances you could classify a racist, classist, sexist or other hate speech meme as inappropiate content. Ensure you are correctly following all the output formats specified.\n"

model = genai.GenerativeModel(model_name="models/gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              safety_settings=safety_settings)

In [9]:
index = 1
trash_index = 1

with open(new_dataset_path, "a", newline="") as file:
    reader = csv.writer(file)
    for filename in os.listdir(noclass_memes):
        if filename.endswith('.jpg'):  # Filter by extension
            # Load image
            img_path = os.path.join(noclass_memes, filename)
            img = Image.open(img_path)
            try:
                # Obtain model output
                response = model.generate_content([system_instruction,img], stream=False)
                response.resolve()
                output = response.text
                print(output)
                
                # Check if the meme is English or Spanish
                language = re.search(r'\*\*(.*?)\*\*', output).group(1)     # **LANGUAGE**
                if language == "English":
                    label_1 = "none"
                else:
                    # If Spanish extract text and first label
                    meme_text = re.search(r'/\*(.*?)\*/', output).group(1)      # /*IMG TEXT*/ 
                    label_1 = re.search(r'\|\|(.*?)\|\|', output).group(1)      # ||LABEL_1||

                # Moving the image to its corresponding directory 
                if label_1 == "hate_speech":
                    label_2 = re.search(r'&&([^&]*)&&', output).group(1)        # &&LABEL_2&&
                    img_dest_path = hs_path + label_2 + "/NEW_IMG_" + str(index) + ".jpg"
                    # Create csv entry
                    reader.writerow(["NEW_IMG_" + str(index), meme_text, label_1, label_2])
                    index+=1
                elif label_1 == "inappropiate_content":
                    img_dest_path = ic_path + "/NEW_IMG_" + str(index) + ".jpg"
                    # Create csv entry
                    reader.writerow(["NEW_IMG_" + str(index), meme_text, label_1, label_1])
                    index+=1
                else:
                    img_dest_path = none_path + "/NEW_IMG_"+ str(trash_index) + ".jpg"
                    trash_index+=1
                shutil.move(img_path, img_dest_path)
            except:
                print("API FAILURE")
            time.sleep(20)

**Spanish**
/*Parece ser que la cerveza contiene hormonas femeninas después de beber suficiente, no sabes conducir, ni puedes parar de hablar*/ 

This meme implies that women talk a lot and cannot drive, perpetuating harmful stereotypes. ||hate_speech|| &&sexism&& 

**English**

/*I AM BLACK AND I WAS THE PRESIDENT OF THE USA, YOU LOOSER*/

The meme makes a statement about Barack Obama's presidency and his race, but it does not express hate or discrimination towards any specific group. The use of the word "looser" (misspelled) could be considered offensive, but it's used in a context of rivalry or competition, not to promote hate. 

||none|| &&N/A&& 

